## Оценка размеров фрагментов в простом RAG

Выбор оптимального размера фрагментов текста критически важен для эффективности поиска в конвейере Retrieval-Augmented Generation (RAG). Наша цель - найти баланс между:
- Точностью поиска (релевантность найденных фрагментов)
- Полнотой информации (достаточный контекст для генерации ответа)

Методика оценки включает следующие этапы:

1. Извлечение текста из PDF-документа
2. Сегментация текста на фрагменты разной длины
3. Векторизация (создание эмбеддингов) для каждого фрагмента
4. Семантический поиск по векторному пространству
5. Генерация ответа на основе найденных фрагментов
6. Оценка качества ответов по двум метрикам:
   - Достоверность (соответствие исходным данным)
   - Релевантность (соответствие вопросу)
7. Сравнительный анализ для разных размеров фрагментов

## Настройка рабочего окружения
Импортируем необходимые Python-библиотеки:

In [1]:
import fitz
import os
import numpy as np
import json
from openai import OpenAI

## Конфигурация клиента OpenAI API
Инициализация подключения к API для:
- Генерации векторных представлений текста (эмбеддингов)
- Формирования итоговых ответов на вопросы

In [ ]:
# Инициализируем клиент OpenAI с базовым URL и API ключом
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # Получаем API ключ из переменных окружения
)

## Экстракция текста из PDF
Используем библиотеку PyMuPDF (fitz) для извлечения текста из документа `AI_Information.pdf`.
Особенности обработки:
- Сохранение структуры абзацев
- Удаление лишних пробелов
- Конкатенация текста со всех страниц

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    Извлекает текстовое содержимое из PDF документа.

    Параметры:
    pdf_path (str): Путь к PDF файлу

    Возвращаемое значение:
    str: Текст документа, объединённый со всех страниц
    """
    # Открываем PDF-файл
    mypdf = fitz.open(pdf_path)
    all_text = ""  # Инициализируем пустую строку для хранения извлечённого текста
    
    # Итерируемся по каждой странице в PDF
    for page in mypdf:
        # Извлекаем текст с текущей страницы и добавляем пробел
        all_text += page.get_text("text") + " "

    # Возвращаем извлечённый текст, очищенный от начальных/конечных пробелов
    return all_text.strip()

# Определяем путь к PDF-файлу
pdf_path = "data/AI_Information.pdf"

# Извлекаем текст из PDF-файла
extracted_text = extract_text_from_pdf(pdf_path)

# Печатаем первые 500 символов извлечённого текста
print(extracted_text[:500])

Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past f


## Сегментация текста на фрагменты
Алгоритм разделения текста с перекрытием (overlap) обеспечивает:
- Сохранение контекста между соседними фрагментами
- Гибкость в выборе размера фрагментов
- Возможность сравнения разных стратегий чанкинга

In [4]:
def chunk_text(text, n, overlap):
    """
    Сегментирует текст на фрагменты с заданным перекрытием.

    Параметры:
    text (str): Исходный текст для сегментации
    n (int): Максимальная длина фрагмента в символах
    overlap (int): Размер перекрытия между соседними фрагментами

    Возвращаемое значение:
    List[str]: Список текстовых фрагментов
    """
    chunks = []  # Инициализируем пустой список для хранения чанков
    for i in range(0, len(text), n - overlap):
        # Добавляем чанк текста от текущего индекса до индекса + размер чанка
        chunks.append(text[i:i + n])
    
    return chunks  # Возвращаем список текстовых чанков

# Определяем различные размеры чанков для оценки
chunk_sizes = [128, 256, 512]

# Создаём словарь для хранения текстовых чанков для каждого размера
text_chunks_dict = {size: chunk_text(extracted_text, size, size // 5) for size in chunk_sizes}

# Печатаем количество созданных чанков для каждого размера
for size, chunks in text_chunks_dict.items():
    print(f"Chunk Size: {size}, Number of Chunks: {len(chunks)}")

Chunk Size: 128, Number of Chunks: 326
Chunk Size: 256, Number of Chunks: 164
Chunk Size: 512, Number of Chunks: 82


## Векторизация текстовых фрагментов
Преобразование текста в векторные представления (эмбеддинги) позволяет:
- Сравнивать семантическое сходство текстов
- Эффективно индексировать и искать информацию
- Использовать математические операции над текстовыми данными

In [5]:
from tqdm import tqdm

def create_embeddings(texts, model="BAAI/bge-en-icl"):
    """
    Создаёт векторные представления для списка текстов.

    Параметры:
    texts (List[str]): Тексты для векторизации
    model (str): Идентификатор модели эмбеддингов

    Возвращаемое значение:
    List[np.ndarray]: Список векторных представлений
    """
    # Создаём эмбеддинги с использованием указанной модели
    response = client.embeddings.create(model=model, input=texts)
    # Преобразуем ответ в список numpy массивов и возвращаем
    return [np.array(embedding.embedding) for embedding in response.data]

# Генерируем эмбеддинги для каждого размера чанков
# Итерируемся по каждому размеру чанков и соответствующим чанкам в text_chunks_dict
chunk_embeddings_dict = {size: create_embeddings(chunks) for size, chunks in tqdm(text_chunks_dict.items(), desc="Generating Embeddings")}

Generating Embeddings: 100%|██████████| 3/3 [00:11<00:00,  3.71s/it]


## Семантический поиск по векторному пространству
Алгоритм поиска основан на:
- Косинусной мере схожести векторов
- Ранжировании результатов по релевантности
- Возможности настройки количества возвращаемых фрагментов (top-k)

In [6]:
def cosine_similarity(vec1, vec2):
    """
    Вычисляет косинусное сходство между векторами.

    Параметры:
    vec1 (np.ndarray): Первый вектор
    vec2 (np.ndarray): Второй вектор

    Возвращаемое значение:
    float: Значение косинусного сходства [-1, 1]
    """

    # Вычисляем скалярное произведение двух векторов
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [7]:
def retrieve_relevant_chunks(query, text_chunks, chunk_embeddings, k=5):
    """
    Находит топ-k наиболее релевантных текстовых чанков.
    
    Аргументы:
    query (str): Пользовательский запрос.
    text_chunks (List[str]): Список текстовых чанков.
    chunk_embeddings (List[np.ndarray]): Эмбеддинги текстовых чанков.
    k (int): Количество возвращаемых топовых чанков.
    
    Возвращает:
    List[str]: Наиболее релевантные текстовые чанки.
    """
    # Генерируем эмбеддинг для запроса - передаём запрос как список и берём первый элемент
    query_embedding = create_embeddings([query])[0]
    
    # Вычисляем косинусную схожесть между эмбеддингом запроса и каждым эмбеддингом чанка
    similarities = [cosine_similarity(query_embedding, emb) for emb in chunk_embeddings]
    
    # Получаем индексы топ-k наиболее схожих чанков
    top_indices = np.argsort(similarities)[-k:][::-1]
    
    # Возвращаем топ-k наиболее релевантных текстовых чанков
    return [text_chunks[i] for i in top_indices]

In [8]:
# Загружаем валидационные данные из JSON-файла
with open('data/val.json') as f:
    data = json.load(f)

# Извлекаем первый запрос из валидационных данных
query = data[3]['question']

# Получаем релевантные чанки для каждого размера чанков
retrieved_chunks_dict = {size: retrieve_relevant_chunks(query, text_chunks_dict[size], chunk_embeddings_dict[size]) for size in chunk_sizes}

# Печатаем найденные чанки для размера чанков 256
print(retrieved_chunks_dict[256])

['AI enables personalized medicine by analyzing individual patient data, predicting treatment \nresponses, and tailoring interventions. Personalized medicine enhances treatment effectiveness \nand reduces adverse effects. \nRobotic Surgery \nAI-powered robotic s', ' analyzing biological data, predicting drug \nefficacy, and identifying potential drug candidates. AI-powered systems reduce the time and cost \nof bringing new treatments to market. \nPersonalized Medicine \nAI enables personalized medicine by analyzing indiv', 'g \npatient outcomes, and assisting in treatment planning. AI-powered tools enhance accuracy, \nefficiency, and patient care. \nDrug Discovery and Development \nAI accelerates drug discovery and development by analyzing biological data, predicting drug \neffica', 'mains. \nThese applications include: \nHealthcare \nAI is transforming healthcare through applications such as medical diagnosis, drug discovery, \npersonalized medicine, and robotic surgery. AI-powered to

## Генерация финального ответа
Используем LLM для формирования ответа на основе:
- Топ-5 наиболее релевантных фрагментов (размер 256 символов)
- Строгого системного промта (отвечать только по контексту)
- Модели Llama-3 с температурой 0 для детерминированности

In [9]:
# Определяем системный промт для AI-ассистента
system_prompt = "You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly from the provided context, respond with: 'I do not have enough information to answer that.'"

def generate_response(query, system_prompt, retrieved_chunks, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    Генерирует AI-ответ на основе найденных чанков.

    Аргументы:
    query (str): Пользовательский запрос.
    retrieved_chunks (List[str]): Список найденных текстовых чанков.
    model (str): AI-модель.

    Возвращает:
    str: Сгенерированный AI-ответ.
    """
    # Объединяем найденные чанки в одну строку контекста
    context = "\n".join([f"Context {i+1}:\n{chunk}" for i, chunk in enumerate(retrieved_chunks)])
    
    # Создаём пользовательский промт, объединяя контекст и запрос
    user_prompt = f"{context}\n\nQuestion: {query}"

    # Генерируем AI-ответ с использованием указанной модели
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # Возвращаем содержимое AI-ответа
    return response.choices[0].message.content

# Генерируем AI-ответы для каждого размера чанков
ai_responses_dict = {size: generate_response(query, system_prompt, retrieved_chunks_dict[size]) for size in chunk_sizes}

# Печатаем ответ для размера чанков 256
print(ai_responses_dict[256])

AI contributes to personalized medicine by analyzing individual patient data, predicting treatment responses, and tailoring interventions. This enables personalized medicine to enhance treatment effectiveness and reduce adverse effects.


## Оценка качества ответов
Метрики оценки:
1. Достоверность (Faithfulness) - соответствие исходным данным
2. Релевантность (Relevancy) - соответствие вопросу
Оценка проводится автоматически с помощью LLM по строгим критериям

In [10]:
# Определяем константы системы оценки
SCORE_FULL = 1.0     # Полное совпадение или полностью удовлетворительно
SCORE_PARTIAL = 0.5  # Частичное совпадение или частично удовлетворительно
SCORE_NONE = 0.0     # Нет совпадения или неудовлетворительно

In [11]:
# Определяем шаблоны строгих промтов для оценки
FAITHFULNESS_PROMPT_TEMPLATE = """
Оцените достоверность ответа AI по сравнению с истинным ответом.
Пользовательский запрос: {question}
Ответ AI: {response}
Истинный ответ: {true_answer}

Достоверность измеряет, насколько хорошо ответ AI соответствует фактам в истинном ответе, без галлюцинаций.

ИНСТРУКЦИИ:
- Оценивайте СТРОГО используя только эти значения:
    * {full} = Полностью достоверно, нет противоречий с истинным ответом
    * {partial} = Частично достоверно, незначительные противоречия
    * {none} = Не достоверно, серьёзные противоречия или галлюцинации
- Возвращайте ТОЛЬКО числовую оценку ({full}, {partial} или {none}) без объяснений или дополнительного текста.
"""

In [12]:
RELEVANCY_PROMPT_TEMPLATE = """
Оцените релевантность ответа AI пользовательскому запросу.
Пользовательский запрос: {question}
Ответ AI: {response}

Релевантность измеряет, насколько хорошо ответ соответствует вопросу пользователя.

ИНСТРУКЦИИ:
- Оценивайте СТРОГО используя только эти значения:
    * {full} = Полностью релевантно, напрямую отвечает на запрос
    * {partial} = Частично релевантно, отвечает на некоторые аспекты
    * {none} = Не релевантно, не отвечает на запрос
- Возвращайте ТОЛЬКО числовую оценку ({full}, {partial} или {none}) без объяснений или дополнительного текста.
"""

In [13]:
def evaluate_response(question, response, true_answer):
        """
        Оценивает качество сгенерированного AI-ответа на основе достоверности и релевантности.

        Аргументы:
        question (str): Оригинальный вопрос пользователя.
        response (str): Оцениваемый сгенерированный AI-ответ.
        true_answer (str): Правильный ответ, используемый как эталон.

        Возвращает:
        Tuple[float, float]: Кортеж, содержащий (оценка_достоверности, оценка_релевантности).
                                                Каждая оценка одна из: 1.0 (полная), 0.5 (частичная), или 0.0 (нет).
        """
        # Форматируем промты для оценки
        faithfulness_prompt = FAITHFULNESS_PROMPT_TEMPLATE.format(
                question=question, 
                response=response, 
                true_answer=true_answer,
                full=SCORE_FULL,
                partial=SCORE_PARTIAL,
                none=SCORE_NONE
        )
        
        relevancy_prompt = RELEVANCY_PROMPT_TEMPLATE.format(
                question=question, 
                response=response,
                full=SCORE_FULL,
                partial=SCORE_PARTIAL,
                none=SCORE_NONE
        )

        # Запрашиваем оценку достоверности у модели
        faithfulness_response = client.chat.completions.create(
               model="meta-llama/Llama-3.2-3B-Instruct",
                temperature=0,
                messages=[
                    {"role": "system", "content": "You are an evaluator that returns only numerical scores (1.0, 0.5 or 0.0) with no additional text."},
                    {"role": "user", "content": faithfulness_prompt}
                ]
        )
        
        # Запрашиваем оценку релевантности у модели
        relevancy_response = client.chat.completions.create(
               model="meta-llama/Llama-3.2-3B-Instruct",
                temperature=0,
                messages=[
                    {"role": "system", "content": "You are an evaluator that returns only numerical scores (1.0, 0.5 or 0.0) with no additional text."},
                    {"role": "user", "content": relevancy_prompt}
                ]
        )

        # Возвращаем оценки достоверности и релевантности
        return (float(faithfulness_response.choices[0].message.content), 
                float(relevancy_response.choices[0].message.content))

# Оцениваем ответы для каждого размера чанков
scores_dict = {}
for size in chunk_sizes:
    if size in [256, 128]:  # Оцениваем только эти размеры для примера
        faithfulness, relevancy = evaluate_response(query, ai_responses_dict[size], data[3]['ideal_answer'])
        scores_dict[size] = (faithfulness, relevancy)
        print(f"Faithfulness Score (Chunk Size {size}): {faithfulness}")
        print(f"Relevancy Score (Chunk Size {size}): {relevancy}\n")

Faithfulness Score (Chunk Size 256): 0.5
Relevancy Score (Chunk Size 256): 0.5


Faithfulness Score (Chunk Size 128): 0.5
Relevancy Score (Chunk Size 128): 0.5
